In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook
# sqlite Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [2]:
from nba_api.stats.endpoints import playercareerstats, drafthistory, commonplayerinfo, playerawards

In [ ]:
# Not used
# cumestatsplayer,draftcombinedrillresults,playerdashboardbyteamperformance, leagueleaders, leaguedashplayerstats, draftcombinenonstationaryshooting, draftcombinestats, commonallplayers,

In [ ]:
# pd.show_versions()

## Static Examples

In [4]:
from nba_api.stats.static import players
# Find players by full name.
players.find_players_by_full_name('james')

# Find players by first name.
players.find_players_by_first_name('lebron')

# Find players by last name.
players.find_players_by_last_name('^(james|love)$')

# Get all players.
# players.get_players()

[{'id': 77115,
  'full_name': 'Aaron James',
  'first_name': 'Aaron',
  'last_name': 'James',
  'is_active': False},
 {'id': 203108,
  'full_name': 'Bernard James',
  'first_name': 'Bernard',
  'last_name': 'James',
  'is_active': False},
 {'id': 202345,
  'full_name': 'Damion James',
  'first_name': 'Damion',
  'last_name': 'James',
  'is_active': False},
 {'id': 77116,
  'full_name': 'Gene James',
  'first_name': 'Gene',
  'last_name': 'James',
  'is_active': False},
 {'id': 1080,
  'full_name': 'Henry James',
  'first_name': 'Henry',
  'last_name': 'James',
  'is_active': False},
 {'id': 1744,
  'full_name': 'Jerome James',
  'first_name': 'Jerome',
  'last_name': 'James',
  'is_active': False},
 {'id': 1629713,
  'full_name': 'Justin James',
  'first_name': 'Justin',
  'last_name': 'James',
  'is_active': True},
 {'id': 2544,
  'full_name': 'LeBron James',
  'first_name': 'LeBron',
  'last_name': 'James',
  'is_active': True},
 {'id': 2229,
  'full_name': 'Mike James',
  'first_nam

## Begin Database Connection (sqlite)

In [ ]:
# Base = declarative_base()
# engine = create_engine('sqlite:///db.sqlite', echo=False)
# conn = engine.connect()
# Create (if not already in existence) the tables associated with our classes.
# Base.metadata.create_all(engine)
# # Create a Session Object to Connect to DB
# # ----------------------------------
# session = Session(bind=engine)

In [ ]:
# Use this to clear out the db
# ----------------------------------
# Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

## List of all players
find specific player

In [11]:
from nba_api.stats.static import players
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))
nba_players[:]
all_players = pd.DataFrame(nba_players)

Number of players fetched: 4501


### Find specific player

In [12]:
## loop to find player
player_name = [player for player in nba_players
                   if player['full_name'] == 'Tim Duncan'][0]
player_name

{'id': 1495,
 'full_name': 'Tim Duncan',
 'first_name': 'Tim',
 'last_name': 'Duncan',
 'is_active': False}

## Get Career Stats by player id number
Get player_id number from get_players above

#### Key: 

* GP: Games Played
* MIN: Minutes Played
* FGM: Field Goals Made
* FGA: Field Goals Attempted
* FG%: Field Goal Percentage
* 3PM: 3 Point Field Goals Made
* 3PA: 3 Point Field Goals Attempted
* 3P%: 3 Point Field Goals Percentage
* FTM: Free Throws Made
* FTA: Free Throws Attempted
* FT%: Free Throw Percentage
* OREB: Offensive Rebounds
* DREB: Defensive Rebounds
* REB: Rebounds
* AST: Assists
* TOV: Turnovers
* STL: Steals
* BLK: Blocks
* PF: Personal Fouls
* DD2: Double Doubles
* TD3: Trible Doubles
* PTS: Points
* YIL: Year in League




example

In [21]:
# Display all columns
pd.set_option('display.max_columns', 500)
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id='1495')
career_df = career.get_data_frames()[0]

career_dict = career_df.to_dict('records')
first_5_years = career_df[0:5]
first_5_years

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1495,1997-98,00,1610612759,SAS,22.0,82,82,3204.0,706,1287,0.549,0,10,0.000,319,482,0.662,274,703,977,224,55,206,279,254,1731
1,1495,1998-99,00,1610612759,SAS,23.0,50,50,1963.0,418,845,0.495,1,7,0.143,247,358,0.690,159,412,571,121,45,126,146,147,1084
2,1495,1999-00,00,1610612759,SAS,24.0,74,74,2875.0,628,1281,0.490,1,11,0.091,459,603,0.761,262,656,918,234,66,165,242,210,1716
3,1495,2000-01,00,1610612759,SAS,25.0,82,82,3174.0,702,1406,0.499,7,27,0.259,409,662,0.618,259,738,997,245,70,192,242,247,1820
4,1495,2001-02,00,1610612759,SAS,26.0,82,82,3329.0,764,1504,0.508,1,10,0.100,560,701,0.799,268,774,1042,307,61,203,263,217,2089


In [24]:
from sklearn import preprocessing

In [5]:
career_df.columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

-----------

#### PySpark databases 
testing

In [ ]:
# import os
# import findspark
# findspark.init()

In [ ]:
# # Start Spark session
# from pyspark.sql import SparkSession
# from pyspark import SparkFiles
# spark = SparkSession.builder.appName("nbaStats").getOrCreate()

In [ ]:
# with open("sample.json", "w") as outfile:  
#     json.dump(career_dict, outfile) 

In [ ]:
# sc = spark.sparkContext
# path = "./sample.json"
# players_json_spark = spark.read.json(path)

In [ ]:
# players_json_spark.printSchema()

In [ ]:
# Creates a temporary view using the DataFrame
# players_json_spark.createOrReplaceTempView("people")

In [ ]:
# players_json_spark.show()

In [ ]:
# # playersDF_spark = spark.range(3).toDF("myCol")
# newRow = players_json_spark
# appended = players_json_spark.union(newRow)
# display(appended)

In [ ]:
# appended.count()

In [ ]:
# def customFunction(row):

#    return (row)

# sample2 = appended.rdd.map(customFunction)
# sample2

In [ ]:
# for ids in all_players_ids: 
#     player_to_pass = ids

In [ ]:
# for f in appended.collect(): 
#         print (f)

In [ ]:
# playersDF_spark.withColumn('newprice', dataframe]).show()

In [ ]:
# otherPeople = spark.read.json(path)
# otherPeople.show()
# otherPeople.select(otherPeople["AST"]).show()

In [ ]:
# Create DataFrame manually
# dataframe = spark.createDataFrame(path, schema=main_df_columns)
# dataframe.show()

------------

## Get first 5 years of each player id.

In [19]:
all_players_ids = list(all_players['id'].values.astype(str))
len(all_players_ids)

4501

In [20]:
all_players_col = ['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS']
len(all_players_col)

27

In [18]:
career = playercareerstats.PlayerCareerStats(player_id=player_to_pass)
career_df = career.get_data_frames()[0]
career_df
first_5_years = career_df[0:5]
first_5_years
main_df = main_df.append(first_5_years, ignore_index = True)
main_df

NameError: name 'player_to_pass' is not defined

In [17]:
# career_df['YIL'] = career_df.groupby(['SEASON_ID']).count()
# career_df

In [72]:
# career_df['YIL'] = career_df.groupby(['PLAYER_ID','SEASON_ID','LEAGUE_ID']).transform('count')
# career_df

In [21]:
pbar = tqdm(total=len(all_players_ids))
try: 
    for ids in all_players_ids: 
        player_to_pass = ids

        career = playercareerstats.PlayerCareerStats(player_id=player_to_pass)
        career_df = career.get_data_frames()[0]
        first_5_years = career_df[0:5]
        main_df = main_df.append(first_5_years, ignore_index = True)
        pbar.update(1)
    pbar.close()
except:
    print("something happened")

  0%|                                                                                 | 4/4501 [00:02<41:38,  1.80it/s]

something happened


In [ ]:
all_players_ids

In [ ]:
five_year_all_players = main_df.sort_values(by='SEASON_ID', ascending=False).copy()
five_year_all_players = five_year_all_players.reset_index(drop=True)
five_year_all_players

In [ ]:
five_year_all_players_to_json = five_year_all_players.to_json(orient='records')

In [ ]:
# SAVE: Player_position
with open(f'./_players_all_data.json', 'w') as fp:
    json.dump(five_year_all_players_to_json, fp)

-------------

# Open Saved JSON

In [3]:
with open('./_players_all_data.json') as json_file:
    data = json.load(json_file)

In [74]:
five_year_all_players = pd.read_json(data)
twenty_years_all_players = five_year_all_players[five_year_all_players['SEASON_ID'] > '1999-98']
twenty_years_all_players

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1629010,2019-20,0,1610612746,LAC,23,42,1.0,476.0,46,136,0.338,19.0,67.0,0.284,11,19,0.579,5.0,55.0,60.0,46,13.0,8.0,26.0,55,122
1,1629640,2019-20,0,1610612759,SAS,20,17,1.0,301.0,53,89,0.596,13.0,22.0,0.591,35,44,0.795,10.0,47.0,57.0,15,14.0,2.0,13.0,28,154
2,1628413,2019-20,0,0,TOT,24,7,0.0,47.0,4,11,0.364,0.0,4.0,0.000,2,4,0.500,3.0,6.0,9.0,0,3.0,2.0,2.0,10,10
3,1628984,2019-20,0,1610612766,CHA,25,63,53.0,2211.0,368,963,0.382,218.0,585.0,0.373,191,233,0.820,42.0,170.0,212.0,471,62.0,15.0,181.0,115,1145
4,1629648,2019-20,0,1610612765,DET,22,10,0.0,53.0,5,20,0.250,2.0,10.0,0.200,0,0,0.000,0.0,4.0,4.0,8,1.0,0.0,2.0,6,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5536,1735,2000-01,0,1610612751,NJN,25,29,0.0,279.0,28,88,0.318,1.0,4.0,0.250,25,34,0.735,35.0,74.0,109.0,16,10.0,11.0,19.0,47,82
5537,2046,2000-01,0,1610612760,SEA,23,78,14.0,1524.0,189,439,0.431,18.0,67.0,0.269,67,91,0.736,72.0,177.0,249.0,63,39.0,20.0,53.0,146,463
5538,1918,2000-01,0,1610612742,DAL,25,4,0.0,8.0,0,4,0.000,0.0,0.0,0.000,0,0,0.000,0.0,2.0,2.0,0,0.0,0.0,0.0,2,0
5539,1536,2000-01,0,1610612751,NJN,23,77,40.0,1663.0,243,572,0.425,52.0,155.0,0.335,97,135,0.719,41.0,167.0,208.0,140,86.0,14.0,130.0,166,635


In [ ]:
import plotly.express as px


fig = px.scatter(twenty_years_all_players, x="PTS", y="SEASON_ID", log_x=True, hover_name="PLAYER_ID", hover_data=["SEASON_ID", "PTS"])

fig.show()

In [ ]:
# five_year_all_players.set_index('SEASON_ID')

In [ ]:
## Save all players data to sqlite db 'all_players'
# all_players.to_sql('all_players', con=engine)

## Retreive all players from sqlite db
# engine.execute("SELECT * FROM all_players").fetchall()